# 1. Overview

In [1]:
import re
import time
import logging

import numpy as np
import pandas as pd

import nltk
from nltk import data
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

from bs4 import BeautifulSoup

import gensim
from gensim.models import Word2Vec

f:\python\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


First, load the dataset. We use pandas to read the tab delimited file.

In [2]:
""""
header=0: indicates that the first line of the file contains column names
"delimiter=\t": indicates that the fields are separated by tabs
quoting=3: tells Python to ignore doubled quotes
"""

labeled_train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)

In [3]:
labeled_train.columns

Index(['id', 'sentiment', 'review'], dtype='object')

As a remainder, the following code shows the first review given by a user. From the review, we could see HTML tags, abbreviations, punctuations. We need to clean the data before putting it to a machine learning model.

In [4]:
labeled_train['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

# 2. Data Cleaning

In this part, we are going to do the text data preprocessing, which mainly includes deleting the punctuations, removing HTML tags, removing non-letters, etc. The following function shows how to finish these data cleaning steps.

In [5]:
def review_to_words(raw_review, remove_stopwords = False ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    try:
        review_text = BeautifulSoup(raw_review).get_text() 
    except TypeError:
        review_text = "None"
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    if remove_stopwords:
        stops = set(stopwords.words("english"))                  
        # 
        # 5. Remove stop words
        words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return words

In [6]:
clean_review = review_to_words( labeled_train["review"][0] )
print(clean_review)

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 

f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Now we could run through all the reviews.

In [7]:
# Get the number of reviews based on the dataframe column size
num_reviews = labeled_train["review"].size

# Initialize an empty list to hold the clean reviews
clean_labeled_train_reviews = []

print('Cleaning the labeled training set movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_labeled_train_reviews.append( review_to_words( labeled_train["review"][i] ) )

f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Cleaning the labeled training set movie reviews...
Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [8]:
# Get the number of reviews based on the dataframe column size
num_reviews = unlabeled_train["review"].size

# Initialize an empty list to hold the clean reviews
clean_unlabeled_train_reviews = []

print('Cleaning the unlabeled training set movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 


for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_unlabeled_train_reviews.append( review_to_words( unlabeled_train["review"][i] ) )

Cleaning the unlabeled training set movie reviews...


f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 115764

Review 2000 of 115764

Review 3000 of 115764

Review 4000 of 115764

Review 5000 of 115764

Review 6000 of 115764

Review 7000 of 115764

Review 8000 of 115764

Review 9000 of 115764

Review 10000 of 115764

Review 11000 of 115764

Review 12000 of 115764

Review 13000 of 115764

Review 14000 of 115764

Review 15000 of 115764

Review 16000 of 115764

Review 17000 of 115764

Review 18000 of 115764

Review 19000 of 115764

Review 20000 of 115764

Review 21000 of 115764

Review 22000 of 115764

Review 23000 of 115764

Review 24000 of 115764

Review 25000 of 115764

Review 26000 of 115764

Review 27000 of 115764

Review 28000 of 115764

Review 29000 of 115764

Review 30000 of 115764

Review 31000 of 115764

Review 32000 of 115764

Review 33000 of 115764

Review 34000 of 115764

Review 35000 of 115764

Review 36000 of 115764

Review 37000 of 115764

Review 38000 of 115764

Review 39000 of 115764

Review 40000 of 115764

Review 41000 of 115764

Review 42000 of 115764

R

In [9]:
# Get the number of reviews based on the dataframe column size
num_reviews = test_data["review"].size

# Initialize an empty list to hold the clean reviews
clean_test_data_reviews = []

print('Cleaning the test dataset movie reviews...')

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 

for i in range(num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_test_data_reviews.append( review_to_words( test_data["review"][i] ) )

Cleaning the test dataset movie reviews...


f:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file f:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



# 3. Construct the Model

In this tutorial, to do the sentiment classification task, we want to first use word embedding to generate the representation of each word in a sentence.

In [10]:
sentences = []

print('Parsing the sentences from the labeled training set....')
start = time.time()

for review in clean_labeled_train_reviews:
    sentences.append(review)
    
print('Parsing the sentences from the unlabeled training set....')

for review in clean_unlabeled_train_reviews:
    sentences.append(review)
    
end = time.time()
duration = end - start

print("Time taken for getting all the text: ", duration, ' seconds')

Parsing the sentences from the labeled training set....
Parsing the sentences from the unlabeled training set....
Time taken for getting all the text:  0.03859972953796387  seconds


Since we have got all the words in the labeled and unlabeled dataset, we could use the word2vec model in [gensim](https://radimrehurek.com/gensim/models/word2vec.html) to generate the word vectors.

In [11]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print("Training model...")
start = time.time()
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

end = time.time()
duration = end - start

print("Total time for training the word vectors: ", duration, " seconds")

2018-03-11 09:15:33,933 : INFO : collecting all words and their counts
2018-03-11 09:15:33,935 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2018-03-11 09:15:34,486 : INFO : PROGRESS: at sentence #10000, processed 2385575 words, keeping 51527 word types
2018-03-11 09:15:35,011 : INFO : PROGRESS: at sentence #20000, processed 4747504 words, keeping 67813 word types
2018-03-11 09:15:35,517 : INFO : PROGRESS: at sentence #30000, processed 7109778 words, keeping 81728 word types
2018-03-11 09:15:36,038 : INFO : PROGRESS: at sentence #40000, processed 9467206 words, keeping 92655 word types
2018-03-11 09:15:36,562 : INFO : PROGRESS: at sentence #50000, processed 11825236 words, keeping 99583 word types
2018-03-11 09:15:37,077 : INFO : PROGRESS: at sentence #60000, processed 14190901 words, keeping 105913 word types
2018-03-11 09:15:37,597 : INFO : PROGRESS: at sentence #70000, processed 16573274 words, keeping 111433 word types
2018-03-11 09:15:38,125 : INFO : PROGRESS: at sentence #80000, processed 18978383 words, keeping 116764 word types
2018-03-11 09:15:38,654 : INFO : PROGRESS: at sentence #90000, processed 21360809 words, 

2018-03-11 09:16:39,018 : INFO : PROGRESS: at 22.03% examples, 474240 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:16:40,022 : INFO : PROGRESS: at 22.46% examples, 474784 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:16:41,026 : INFO : PROGRESS: at 22.88% examples, 475309 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:16:42,054 : INFO : PROGRESS: at 23.32% examples, 475883 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:16:43,061 : INFO : PROGRESS: at 23.75% examples, 476595 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:16:44,070 : INFO : PROGRESS: at 24.16% examples, 476905 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:16:45,074 : INFO : PROGRESS: at 24.54% examples, 476710 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:16:46,088 : INFO : PROGRESS: at 24.91% examples, 476083 words/s, in_qsize 8, out_qsize 1
2018-03-11 09:16:47,103 : INFO : PROGRESS: at 25.30% examples, 475807 words/s, in_qsize 8, out_qsize 1
2018-03-11 09:16:48,120 : INFO : PROGRESS: at 25.70% examples, 475887 wor

2018-03-11 09:17:59,961 : INFO : PROGRESS: at 51.40% examples, 454926 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:18:00,964 : INFO : PROGRESS: at 51.75% examples, 454784 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:18:01,974 : INFO : PROGRESS: at 52.11% examples, 454619 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:18:02,994 : INFO : PROGRESS: at 52.46% examples, 454320 words/s, in_qsize 8, out_qsize 1
2018-03-11 09:18:04,012 : INFO : PROGRESS: at 52.84% examples, 454346 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:18:05,032 : INFO : PROGRESS: at 53.19% examples, 454103 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:18:06,071 : INFO : PROGRESS: at 53.55% examples, 453868 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:18:07,099 : INFO : PROGRESS: at 53.89% examples, 453559 words/s, in_qsize 8, out_qsize 2
2018-03-11 09:18:08,105 : INFO : PROGRESS: at 54.23% examples, 453270 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:18:09,125 : INFO : PROGRESS: at 54.57% examples, 453001 wor

2018-03-11 09:19:21,224 : INFO : PROGRESS: at 79.24% examples, 440895 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:19:22,225 : INFO : PROGRESS: at 79.62% examples, 441030 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:19:23,224 : INFO : PROGRESS: at 80.00% examples, 441138 words/s, in_qsize 8, out_qsize 1
2018-03-11 09:19:24,234 : INFO : PROGRESS: at 80.39% examples, 441236 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:19:25,248 : INFO : PROGRESS: at 80.78% examples, 441440 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:19:26,290 : INFO : PROGRESS: at 81.18% examples, 441529 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:19:27,305 : INFO : PROGRESS: at 81.57% examples, 441664 words/s, in_qsize 7, out_qsize 0
2018-03-11 09:19:28,320 : INFO : PROGRESS: at 81.96% examples, 441793 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:19:29,337 : INFO : PROGRESS: at 82.36% examples, 441894 words/s, in_qsize 8, out_qsize 0
2018-03-11 09:19:30,343 : INFO : PROGRESS: at 82.74% examples, 441981 wor

Total time for training the word vectors:  285.1712746620178  seconds


We have got the word vector model. To load this model, you could use the following code:

```Python
model = Word2Vec.load("300features_40minwords_10context")
```

In [17]:
model = Word2Vec.load("300features_40minwords_10context")

2018-03-11 11:07:51,047 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-03-11 11:07:52,859 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-03-11 11:07:52,861 : INFO : setting ignored attribute syn0norm to None
2018-03-11 11:07:52,862 : INFO : setting ignored attribute cum_table to None
2018-03-11 11:07:52,864 : INFO : loaded 300features_40minwords_10context


In [54]:
def makeFeatureVec(sentence, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((300,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    # Loop over each word in the review
    for word in sentence:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [55]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter % 1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[int(counter),] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [57]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
print("Creating average feature vecs for training reviews")

trainDataVecs = getAvgFeatureVecs(clean_labeled_train_reviews, model, 300)

print("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs(clean_test_data_reviews, model, 300)

Creating average feature vecs for training reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Revi

In [61]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, labeled_train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test_data["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
